# Make figures

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import contextily as ctx
from shapely import wkt
from shapely.geometry import Polygon, Point
import geoutils as gu
import numpy as np
from geopy.geocoders import Nominatim

In [ ]:
# define some paths in directory for convenience
data_path = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites'
figures_path = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow/figures'

## Study sites map

In [ ]:
# Set up figure
fontsize = 12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif': "Arial"})
fig, ax = plt.subplots(2,2, figsize=(10,10))
ax = ax.ravel()
crs = "EPSG:32611"

# Define site-specific details
info_dict = {
    "MCS": {
        "refdem_fn": os.path.join(data_path, "MCS", 'refdem', "MCS_REFDEM_WGS84.tif"),
        "SNOTEL_fn": os.path.join(data_path, "MCS", 'snotel', 'MCS_SNOTEL_site_info.csv'),
        "DEM_fns": [
            os.path.join(data_path, "MCS", "20240420", "MCS_20240420_DEM.tif"),
            os.path.join(data_path, "MCS", "20241003", "MCS_20241003_DEM.tif")
        ],
        "site_name_display": "Mores Creek Summit"
    },
    "JacksonCreek": {
        "refdem_fn": os.path.join(data_path, "JacksonCreek", 'refdem', "USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered.tif"),
        "SNOTEL_fn": os.path.join(data_path, "JacksonCreek", 'snotel', 'JacksonCreek_SNOTEL_site_info.csv'),
        "DEM_fns": [
            os.path.join(data_path, "JacksonCreek", "20240420", "JacksonCreek_20240420_DEM.tif")
        ],
        "site_name_display": "Jackson Peak"
    },
    "Banner": {
        "refdem_fn": os.path.join(data_path, "Banner", 'refdem', "Banner_REFDEM_WGS84.tif"),
        "SNOTEL_fn": os.path.join(data_path, "Banner", 'snotel', 'Banner_SNOTEL_site_info.csv'),
        "DEM_fns": [
            os.path.join(data_path, "Banner", "20240419-1", "Banner_20240419-1_DEM.tif"),
            os.path.join(data_path, "Banner", "20240419-2", "Banner_20240419-2_DEM.tif")
        ],
        "site_name_display": "Banner Summit"
    },
}

def bounds_to_polygon(bounds):
    return Polygon([[bounds.left, bounds.bottom],
                    [bounds.right, bounds.bottom],
                    [bounds.right, bounds.top],
                    [bounds.left, bounds.top],
                    [bounds.left, bounds.bottom]])
    
# iterate over site names
labels = ['a', 'b', 'c', 'd']
for i, site_name in enumerate(list(info_dict.keys())):
    
    # Reference DEM
    refdem_fn = info_dict[site_name]['refdem_fn']
    refdem = gu.Raster(refdem_fn).reproject(crs=crs)
    refdem_poly = bounds_to_polygon(refdem.bounds)
    ax[0].plot(*refdem_poly.exterior.coords.xy, color='#fc8d62', linewidth=1, label='_nolegend')
    ax[i+1].plot(*refdem_poly.exterior.coords.xy, color='#fc8d62', linewidth=2, label='Reference DTM')
    ax[0].text(refdem.bounds.left-5e3, (refdem.bounds.top + refdem.bounds.bottom)/2 -2e3,
               labels[i+1], fontweight='bold', color='w')
    
    # DEMs
    for j, dem_fn in enumerate(list(info_dict[site_name]['DEM_fns'])):
        dem = gu.Raster(dem_fn)
        dem_poly = bounds_to_polygon(dem.bounds)
        ax[i+1].plot(*dem_poly.exterior.coords.xy, color='#66c2a5', linewidth=2, label='SkySat DEM')
    
    # SNOTEL site info
    snotel_fn = info_dict[site_name]['SNOTEL_fn']
    snotel = pd.read_csv(snotel_fn)
    snotel['geometry'] = snotel['geometry'].apply(wkt.loads)
    snotel_gdf = gpd.GeoDataFrame(snotel, crs='EPSG:4326')
    snotel_gdf = snotel_gdf.to_crs(dem.crs)
    ax[i+1].plot(*snotel_gdf['geometry'].values[0].coords.xy, '*', markersize=15, 
                 markerfacecolor='b', markeredgecolor='w', linewidth=0.3, label="SNOTEL site")
    
    # title
    ax[i+1].set_title(f"{labels[i+1]}) {info_dict[site_name]['site_name_display']}")
    # adjust axes
    ax[i+1].set_xticks([])
    ax[i+1].set_yticks([])
    ax[i+1].set_xlim(refdem.bounds.left-2e3, refdem.bounds.right+2e3)
    ax[i+1].set_ylim(refdem.bounds.bottom-2e3, refdem.bounds.top+2e3)
    # basemap
    ctx.add_basemap(ax=ax[i+1], source=ctx.providers.USGS.USImagery, attribution=False, crs=dem.crs)
    
# Load and plot HWY21
hwy21_fn = os.path.join(data_path, '..', 'ITD_Functional_Class', 'ITD_HWY_21.shp')
hwy21 = gpd.read_file(hwy21_fn).to_crs(crs)
for i, axis in enumerate(ax):
    for j, geom in enumerate(hwy21.geometry[0].geoms):
        if (i==0) & (j==0):
            label='HWY 21'
        else:
            label='_nolegend'
        axis.plot(*geom.coords.xy, color='#bdbdbd', label=label)
    
# legend
handles0, labels0 = ax[0].get_legend_handles_labels()
handles2, labels2 = ax[2].get_legend_handles_labels()
fig.legend(handles0+handles2, labels0+labels2, loc='upper center', ncols=len(labels0+labels2), frameon=False)

# ax[0] adjustments
ax[0].set_title('a) All sites')
ax[0].set_xticks(ax[0].get_xticks())
ax[0].set_xticklabels(np.divide(ax[0].get_xticks(), 1e3).astype(int).astype(str))
ax[0].set_yticks(ax[0].get_yticks())
ax[0].set_yticklabels(np.divide(ax[0].get_yticks(), 1e3).astype(int).astype(str))
ax[0].set_xlabel('Easting [km]')
ax[0].set_ylabel('Northing [km]')
ax[0].set_xlim(555e3, 655e3)
ax[0].set_ylim(4820e3, 4920e3)

# Add a couple city names
geolocator = Nominatim(user_agent="My-Map")
cities = ["Boise", "Idaho City"]
df = pd.DataFrame(columns=['city', 'geometry'])
for city in cities:
    location = geolocator.geocode(city, addressdetails=True)
    df = pd.concat([df, pd.DataFrame({'city': [city],
                                      'geometry': Point(location.longitude, location.latitude)})])
gdf = gpd.GeoDataFrame(df, crs='EPSG:4326')
gdf = gdf.to_crs((crs)).reset_index(drop=True)
for city in cities:
    ax[0].text(gdf.loc[gdf['city']==city].geometry.values[0].coords.xy[0][0], 
               gdf.loc[gdf['city']==city].geometry.values[0].coords.xy[1][0],
               city, fontsize=fontsize+1, color='#969696', fontweight='bold', style='italic', ha='center')

# basemap
ctx.add_basemap(ax=ax[0], source=ctx.providers.USGS.USImagery, attribution=False, crs=dem.crs)

fig.tight_layout(pad=3)
plt.show()
    
# Save figure
fig_fn = os.path.join(figures_path, "study_sites.png")
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)
    

In [ ]:
geolocator.geocode("Robie Creek", addressdetails=True)